In [2]:
import sys
import os

notebook_path = os.getcwd() 
project_root = os.path.dirname(notebook_path)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [11]:
import NN_models
import Filters
import utils
import Systems
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateKalmanNet import StateKalmanNet

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [13]:
state_dim = 1
obs_dim = 1

# --- Reálný systém ("Ground Truth") ---
F_true = torch.tensor([[0.5]])
H_true = torch.tensor([[1.0]])
Q_true = torch.tensor([[0.1]])
R_true = torch.tensor([[0.1]])
Ex0_true = torch.tensor([[1.0]])  # Střední hodnota počátečního stavu
P0_true = torch.tensor([[1.2]])   # Počáteční kovariance

# --- Model systému ---
F_model = F_true
H_model = H_true
Q_model = Q_true
R_model = R_true
Ex0_model = torch.tensor([[0.5]])  # Schválně jiná střední hodnota
P0_model = torch.tensor([[1.5]])   # Schválně jiná počáteční kovariance


sys_true = Systems.DynamicSystem(
    state_dim=state_dim,
    obs_dim=obs_dim,
    Ex0=Ex0_true,
    P0=P0_true,
    Q=Q_true,
    R=R_true,
    F=F_true,
    H=H_true,
    device=device
)

sys_model = Systems.DynamicSystem(
    state_dim=state_dim,
    obs_dim=obs_dim,
    Ex0=Ex0_model,
    P0=P0_model,
    Q=Q_model,
    R=R_model,
    F=F_model,
    H=H_model,
    device=device
)

print("\nInicializace systémů dokončena.")
print(f"Reálný systém: f(x) = {sys_true.F.item():.2f}*x, h(x) = {sys_true.H.item():.2f}*x")
print(f"Model systému: f(x) = {sys_model.F.item():.2f}*x, h(x) = {sys_model.H.item():.2f}*x")


Inicializace systémů dokončena.
Reálný systém: f(x) = 0.50*x, h(x) = 1.00*x
Model systému: f(x) = 0.50*x, h(x) = 1.00*x


In [14]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 200      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 5000   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 500    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 16         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [15]:
kalman_filter = Filters.ExtendedKalmanFilter(sys_model)
y_first_trajectory = y_test[0] 
x_true_first_trajectory = x_test[0]

kf_results = kalman_filter.process_sequence(y_first_trajectory)

x_hat_kf = kf_results['x_filtered']
P_hat_kf = kf_results['P_filtered']

print("\nZpracování dokončeno.")
print(f"Tvar odhadnutých stavů: {x_hat_kf.shape}")
print(f"Tvar skutečných stavů:  {x_true_first_trajectory.shape}")

mse_kf = torch.nn.functional.mse_loss(x_hat_kf, x_true_first_trajectory)
print(f"MSE pro EKF na první trajektorii: {mse_kf.item():.4f}")


Zpracování dokončeno.
Tvar odhadnutých stavů: torch.Size([200, 1])
Tvar skutečných stavů:  torch.Size([200, 1])
MSE pro EKF na první trajektorii: 0.0543


In [ ]:
state_knet = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=10).to(device)
utils.train_state_KalmanNet(
    model=state_knet, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=100, 
    lr=1e-4,
    early_stopping_patience=15
)

/home/luky/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [5/100], Train Loss: 0.067964, Val Loss: 0.060342
Epoch [10/100], Train Loss: 0.057707, Val Loss: 0.055919


In [ ]:
plt.figure(figsize=(15, 7))
plt.title("Porovnání odhadů: KalmanNet vs. Klasický Kalmanův Filtr", fontsize=16)

x_true_plot = x_test.squeeze().numpy()
y_meas_plot = y_test.squeeze().numpy()
x_hat_knet_plot = x_hat_knet.squeeze().numpy()
x_hat_kf_plot = x_hat_kf.squeeze().numpy()

plt.plot(x_true_plot, 'k-', linewidth=2, label="Skutečný stav (Ground Truth)")
plt.plot(y_meas_plot, 'r.', markersize=4, alpha=0.6, label="Měření")
plt.plot(x_hat_knet_plot, 'g--', linewidth=2.5, label=f"Odhad KalmanNet (MSE={mse_knet.item():.4f})")
plt.plot(x_hat_kf_plot, 'b--', linewidth=2.5, label=f"Odhad Klasický KF (MSE={mse_kf.item():.4f})")

plt.xlabel("Časový krok", fontsize=12)
plt.ylabel("Hodnota", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)
plt.show()